In [1]:
import numpy as np
import pandas as pd
import os
#print(os.listdir("../input"))
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from PIL import Image
%matplotlib inline

sys.path.append('rxrx1-utils')
import rxrx.io as rio

import keras
from keras import utils
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn import metrics
import tensorflow as tf

Using TensorFlow backend.


In [2]:
with tf.Session() as sess: 
    devices = sess.list_devices()
    print(devices)

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 3951163469437716412), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3728998500473658934), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 2129640441876517978), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 7380765901, 3696587968046882943)]


In [3]:
train_df = pd.read_csv('../data/train.csv')

In [4]:
train_df

,id_code,experiment,plate,well,sirna
0,HEPG2-01_1_B03,HEPG2-01,1,B03,513
1,HEPG2-01_1_B04,HEPG2-01,1,B04,840
2,HEPG2-01_1_B05,HEPG2-01,1,B05,1020
3,HEPG2-01_1_B06,HEPG2-01,1,B06,254
4,HEPG2-01_1_B07,HEPG2-01,1,B07,144
5,HEPG2-01_1_B08,HEPG2-01,1,B08,503
6,HEPG2-01_1_B09,HEPG2-01,1,B09,188
7,HEPG2-01_1_B10,HEPG2-01,1,B10,700
8,HEPG2-01_1_B11,HEPG2-01,1,B11,1100
9,HEPG2-01_1_B12,HEPG2-01,1,B12,611


In [44]:
def parse_all_train():
    dataset = 'train'
    experiments = train_df['experiment'].values
    plates = train_df['plate'].values
    wells = train_df['well'].values
    sites = np.array([1,2])
    channels=rio.DEFAULT_CHANNELS
    
    for site in sites:
        for k in range(0, 36000, 1000):
            print("Current cycle: ", k)
            site_img = np.empty((1000, 512, 512, 6), dtype=np.float16)
            for i  in tqdm_notebook(range(k, k+site_img.shape[0])):
                j=0
                for channel in channels:
                    path = f'../data/{dataset}/{experiments[i]}/Plate{plates[i]}/{wells[i]}_s{site}_w{channel}.png'
                    site_img[j, :, :, channel-1] = Image.open(path)
                j+=1
                    
            #normalize here
            site_img /= 255

            #get targets to write out
            y_train = train_df['sirna'].values[k:k+site_img.shape[0]]
            y_train = utils.to_categorical(y_train)

            print("x_train shape: ", site_img.shape)
            print("y_train shape: ", y_train.shape)

            print("Writing to file...")
            np.save(f'../data/train_parsed/x_train_{k}', site_img, allow_pickle=True)
            np.save(f'../data/train_parsed/y_train_{k}', y_train, allow_pickle=True)

In [ ]:
parse_all_train()

Current cycle:  0


x_train shape:  (1000, 512, 512, 6)
y_train shape:  (1000, 1108)
Writing to file...
Current cycle:  1000


x_train shape:  (1000, 512, 512, 6)
y_train shape:  (1000, 1107)
Writing to file...
Current cycle:  2000


x_train shape:  (1000, 512, 512, 6)
y_train shape:  (1000, 1108)
Writing to file...
Current cycle:  3000


In [ ]:
x_train.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(x_train[0,:,:,0])
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(x_train[10000,:,:,0])
plt.show()

In [16]:
y_train = train_df['sirna'].values
y_train = utils.to_categorical(y_train)
print(y_train.shape)

(36515, 1108)


In [ ]:
np.save('../data/x_train_0', x_train, allow_pickle=True)
np.save('../data/y_train_0', y_train, allow_pickle=True)

-------------------

In [ ]:
x_train = np.load('../data/x_train.npy', allow_pickle=True)
y_train = np.load('../data/y_train.npy', allow_pickle=True)

In [ ]:
x_train = x_train.astype('float32')

In [17]:
x_train /= 255

In [18]:
print(np.amax(x_train))
print(np.amin(x_train))

1.0
0.0


## model

In [ ]:
# Training parameters
batch_size = 1
epochs = 2
data_augmentation = False
n = 1
depth = n * 9 + 2
version = 2
# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

In [ ]:
# Input image dimensions.
input_shape = x_train.shape[1:]

model = resnet_v2(input_shape=input_shape, depth=depth)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

model.summary()
print(model_type)

In [19]:
keras.backend.clear_session()

In [20]:
def test_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(8, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    x = Conv2D(8, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = Conv2D(16, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding='same', strides=2)(x)
    x = Conv2D(16, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = Conv2D(32, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding='same', strides=2)(x)
    x = Conv2D(32, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding='same', strides=2)(x)
    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(1108, activation='relu', kernel_initializer='he_normal')(y)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

In [21]:
model = test_model(x_train.shape[1:])
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 512, 512, 8)       80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 512, 512, 8)       584       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 16)      1168      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 256, 256, 16)      2320      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 32)      9248      
__________

In [22]:
model.fit(x_train, y_train[0:20_000],
          batch_size=2,
          epochs=20,
          validation_split=0.1,
          shuffle=True,
          callbacks=[],
          verbose=True)

# Score trained model.
# scores = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', scores[0])
# print('Test mean_absolute_error:', scores[1])
# print('Test mean_squared_error:', scores[2])

Train on 18000 samples, validate on 2000 samples
Epoch 1/20
18000/18000 [==============================] - 185s 10ms/step - loss: 16.0490 - categorical_accuracy: 1.0000e-03 - val_loss: 16.0879 - val_categorical_accuracy: 1.0000e-03
Epoch 2/20
14802/18000 [=======================>......] - ETA: 30s - loss: 16.0603 - categorical_accuracy: 6.0803e-04

KeyboardInterrupt: 

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):

    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v2(input_shape, depth, num_classes=10):
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=4)(x)
    y = Flatten()(x)
    outputs = Dense(1108, activation='relu', kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        #self.val_losses = []
        
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        #self.val_losses.append(logs.get('val_loss'))